## Realizamos las predicciones del dataset de evaluacion con un Multilayer Perceptron + Word2Vec para 'Type'

### Cargamos Dataset de Entrenamiento y de Evaluacion

In [4]:
import pandas as pd
import numpy as np
import os
# Ruta de lectura
ruta = r"C:\Users\uzgre\Escritorio"
archivo = os.path.join(ruta, "Train_Limpio.csv") 

with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data_train = pd.read_csv(f)

archivo = os.path.join(ruta, "Test_Limpio.csv")
with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data_test = pd.read_csv(f)

### Representacion Word2Vec

In [5]:
from gensim.models import Word2Vec
def Representacion_word2vec(dataframe, model=None, vector_size=100, window=5, min_count=5, sg=1, epochs=10, train_model=True):
    textos = dataframe['Texto_Limpio'].fillna("").apply(str.split).values

    if train_model:
        model = Word2Vec(
            sentences=textos, 
            vector_size=vector_size, 
            window=window, 
            min_count=min_count, 
            sg=sg,
            workers=4,
            epochs=epochs
        )

    def obtener_vector_promedio(tokens):
        vectores = [model.wv[word] for word in tokens if word in model.wv]
        return np.mean(vectores, axis=0) if vectores else np.zeros(model.vector_size)

    vectores_documento = np.array([obtener_vector_promedio(tokens) for tokens in textos])
    
    return vectores_documento, model

### ENTRENAR MLP CON TODO EL TRAIN

In [6]:
# Entrenar Word2Vec en Data_train
X_train_vecs, w2v_model = Representacion_word2vec(Data_train, vector_size=100, window=3, min_count=5, sg=1, epochs=10, train_model=True)

# Representar Data_test usando el mismo modelo entrenado
X_test_vecs, _ = Representacion_word2vec(Data_test, model=w2v_model, train_model=False)

In [ ]:
from sklearn.neural_network import MLPClassifier
# ===============================
y_train = Data_train['Type']

mlp = MLPClassifier(
    hidden_layer_sizes=(256, 128, 64),
    activation='relu',
    solver='adam',
    alpha=1e-4,
    learning_rate='adaptive',
    max_iter=300,
    random_state=42,
    early_stopping=False  # Ahora sí, todo el train sin split
)

mlp.fit(X_train_vecs, y_train)

# ===============================
# PREDICCIONES FINALES
# ===============================
y_test_pred = mlp.predict(X_test_vecs)

# ===============================
# GENERAR ARCHIVO .TXT ENTREGABLE
# ===============================
output_file = "CorpusChristi_Type_Run.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for idx, row in Data_test.iterrows():
        instance_id = row['ID']
        predicted_class = y_test_pred[idx]  # Suponiendo que ya está en 1-5 o similar según tu variable 'Type'
        
        output_line = f"rest-mex  {instance_id}  {predicted_class}\n"
        f.write(output_line)

print(f"Archivo '{output_file}' generado exitosamente.")


Archivo 'CorpusChristi_Type_Run.txt' generado exitosamente.


In [12]:
Data_test['Texto_Leido'][5]

'Una buena manera de pasar la mañana Esto era las únicas ruinas visitado que no nos guía, pero nos pareció que no era muy necesario. Escuchamos cosas y piezas de otras excursiones que pasan, y el parque fue principalmente sobre las vistas, que no puede ser mejor.'